## Setup


Ensure you have the following packages and python libraries installed 

```code
pip install teradataml==17.0.0.4 aoa==6.1.0 pandas==1.1.5
```

The remainder of the notebook runs through the following steps

- Connect to Vantage
- Create DDLs
- Import Data


In [1]:
import getpass
import logging
import sys
import urllib

from aoa import create_features_stats_table
from teradataml import create_context
from teradataml import get_context
from teradataml import copy_to_sql, DataFrame
from teradatasqlalchemy.types import *
from teradataml.dataframe.data_transfer import read_csv

import pandas as pd

In [3]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

host = "tdprd2.td.teradata.com"
username = "tf255023"
password = getpass.getpass("Password:")

engine = create_context(host=host, username=username, password=urllib.parse.quote(password), logmech="TDNEGO")

Password:········


### Create DDLs

Create the following tables 

- aoa_feature_metadata 
- aoa_byom_models
- pima_patient_predictions

`aoa_feature_metadata` is used to store the profiling metadata for the features so that we can consistently compute the data drift and model drift statistics. This table can also be created via the CLI by executing 

```bash
aoa feature create-stats-table -m <features-db>.<features-table>
```

`pima_patient_predictions` is used for storing the predictions of the model scoring for the demo use case

In [4]:
# Note: assuming we are using user database for training. If another database (e.g. datalab) is being used, please update.
# Also note, if a shared datalab is being used, only one user should execute the following DDL/DML commands
database = 'ADKCSV2_HEALTHCARE'

In [2]:
create_features_stats_table(f"{database}.hf_automl_byom_modelops_feature_metadata")

get_context().execute(f"""
CREATE MULTISET TABLE {database}.hf_automl_byom_modelops_models
    (
        model_version VARCHAR(255),
        model_id VARCHAR(255),
        model_type VARCHAR(255),
        project_id VARCHAR(255),
        deployed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        model BLOB(2097088000)
    )
    UNIQUE PRIMARY INDEX ( model_version );
""")

get_context().execute(f"""
CREATE MULTISET TABLE {database}.hf_automl_byom_modelops_predictions
    (
        job_id VARCHAR(255),
        PatientId BIGINT,
        HasDiabetes BIGINT,
        json_report CLOB(1048544000) CHARACTER SET UNICODE
    )
    PRIMARY INDEX ( job_id );
""")

### Import Data

Create and import the data for the following two tables

- pima_patient_features
- pima_patient_diagnoses
- aoa_feature_metadata

`pima_patient_features` contains the features related to the patients medical history.

`pima_patient_diagnoses` contains the diabetes diagnostic results for the patients.

`aoa_feature_metadata` contains the feature statistics data for the `pima_patient_features` and `pima_patient_diagnoses`

Note the `pima_patient_feature` can be populated via the CLI by executing 

```bash
aoa feature compute-stats -s <data-db>.PIMA -m <features-db>.<features-table> -t continuous -c numtimesprg,plglcconc,bloodp,skinthick,twohourserins,bmi,dipedfunc,age 
```

In [12]:
hf_data = pd.read_sql("SELECT * FROM ADKCSV2_HEALTHCARE.HF_DATA;", engine)
hf_data.head()

,MBR_ID,DEM_age,DM_fml,DM_ml,DG_bdmnl hrn_m6_12,DG_bdmnl hrn_mlt_12,DG_bdmnl pn nd thr dgstv/bdmn sgns nd symptms_m1_3,DG_bdmnl pn nd thr dgstv/bdmn sgns nd symptms_m3_6,DG_bdmnl pn nd thr dgstv/bdmn sgns nd symptms_m6_12,DG_bdmnl pn nd thr dgstv/bdmn sgns nd symptms_mlt_12,...,PRC_Trtmnt f Lsns: njctn_mlt_12,PRC_0001-0005 Crnvrs Srvcs_m1_3,PRC_0001-0005 Crnvrs Srvcs_m3_6,PRC_0001-0005 Crnvrs Srvcs_m6_12,PRC_0001-0005 Crnvrs Srvcs_mlt_12,PRC_V2020-V2025 Frms_m6_12,PRC_V2020-V2025 Frms_mlt_12,PRC_V2700-V2799 Mscllns_m6_12,PRC_V2700-V2799 Mscllns_mlt_12,trgt
0,27864619,55,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,27947859,51,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23404779,64,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,23425743,52,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,26513327,71,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [10]:
hf_data_train = hf_data.loc[:round(0.9 * len(hf_data))-1]
hf_data_test = hf_data.loc[round(0.9 * len(hf_data)):]

In [6]:
hf_data_train.to_csv('data/hf_data_train.csv', index=False)
hf_data_test.to_csv('data/hf_data_test.csv', index=False)

In [16]:
read_csv(filepath="data/hf_data_train.csv", schema_name='ADKCSV2_HEALTHCARE', table_name='HF_DATA_AUTOML_TRAIN', if_exists='append', use_fastload=True, catch_errors_warnings=True)

read_csv(filepath="data/hf_data_test.csv", schema_name='ADKCSV2_HEALTHCARE', table_name='HF_DATA_AUTOML_TEST', if_exists='append', use_fastload=True, catch_errors_warnings=True)

# we can compute this from the CLI also - but lets import pre-computed for now.
# df = pd.read_csv("data/aoa_feature_metadata.csv")
# copy_to_sql(df=df, 
#             table_name="aoa_feature_metadata",     
#             schema_name=database,
#             if_exists="append")

(          DEM_age  DM_fml  DM_ml  DG_bdmnl hrn_m6_12  DG_bdmnl hrn_mlt_12  DG_bdmnl pn nd thr dgstv/bdmn sgns nd symptms_m1_3  DG_bdmnl pn nd thr dgstv/bdmn sgns nd symptms_m3_6  DG_bdmnl pn nd thr dgstv/bdmn sgns nd symptms_m6_12  DG_bdmnl pn nd thr dgstv/bdmn sgns nd symptms_mlt_12  DG_bnrml fndngs wtht dgnss_m1_3  DG_bnrml fndngs wtht dgnss_mlt_12  DG_ct nd nspcfd rnl flr_m1_3  DG_ct nd nspcfd rnl flr_m3_6  DG_ct nd nspcfd rnl flr_m6_12  DG_ct nd nspcfd rnl flr_mlt_12  DG_ct brnchts_m6_12  DG_ct brnchts_mlt_12  DG_ct mycrdl nfrctn_m1_3  DG_ct mycrdl nfrctn_m6_12  DG_lchl-rltd dsrdrs_m6_12  DG_lchl-rltd dsrdrs_mlt_12  DG_nxty nd fr-rltd dsrdrs_m6_12  DG_nxty nd fr-rltd dsrdrs_mlt_12  DG_rtc; prphrl; nd vscrl rtry nrysms_m3_6  DG_rtc; prphrl; nd vscrl rtry nrysms_mlt_12  DG_plstc nm_m1_3  DG_plstc nm_m3_6  DG_plstc nm_mlt_12  DG_sthm_m1_3  DG_sthm_m6_12  DG_Bctrl nfctns_m3_6  DG_Bngn nplsms_m1_3  DG_Bngn nplsms_m3_6  DG_Bngn nplsms_m6_12  DG_Bngn nplsms_mlt_12  DG_Blry trct dss_m1_3 

## ModelOps UI

#### Add Project

- create project
   - Details
      - Name: Demo {your-name}
      - Description: ModelOps Demo
      - Group: {your-name}
      - Path: https://github.com/Teradata/modelops-demo-models 
      - Credentials: No Credentials
      - Branch: master
      - Save And Continue
   - Service Connection
      - Skip for now
   - Personal Connection
      - Name: Vantage Personal {your-name}
      - Description: Vantage Demo Env
      - Host: {your-host}
      - Database: {your-db}
      - VAL Database: {your-val-db}
      - BYOM Database: (your-byom-db}
      - Login Mech: TDNEGO
      - Username/Password
      
    
#### Add Datasets

- create dataset template
  - Catalog
     - Name: PIMA
     - Description: PIMA Diabetes
     - Feature Catalog: Vantage
     - Database: {your-db}
     - Table: aoa_feature_metadata
  - Features
     - Query: `SELECT * FROM {your-db}.pima_patient_features`
     - Entity Key: PatientId
     - Features: NumTimesPrg, PlGlcConc, BloodP, SkinThick, TwoHourSerIns, BMI, DiPedFunc, Age
  - Entity & Target
     - Query: `SELECT * FROM {your-db}.pima_patient_diagnoses`
     - Entity Key: PatientId
     - Target: HasDiabetes
  - Predictions
     - Database: {your-db}
     - Table: pima_patient_predictions
     - Entity Selection: `SELECT * FROM pima_patient_features WHERE patientid MOD 5 = 0`
     - BYOM Target Column: `CAST(CAST(json_report AS JSON).JSONExtractValue('$.predicted_HasDiabetes') AS INT)`
     
    
- create training dataset
   - Basic
      - Name: Train
      - Description: Training dataset
      - Scope: Training
   - Entity & Target
      - Query: `SELECT * FROM {your-db}.pima_patient_diagnoses WHERE patientid MOD 5 <> 0`
   
- create evaluation dataset
   - Basic
      - Name: Evaluate
      - Description: Evaluation dataset
      - Scope: Evaluation
   - Entity & Target
      - Query: `SELECT * FROM {your-db}.pima_patient_diagnoses WHERE patientid MOD 5 = 0`
    

#### Model Lifecycle

- Python Diabetes Prediction
   - Train
   - Evaluate
   - Review evaluation report
   - Approve 
   - Deploy 
   - Deployments/executions
   - Retire
- R Diabetes Prediction
   - Train
   - Evaluate
   - Review evaluation report
   - Approve 
   - Deploy 
   - Deployments/executions
   - Retire
- BYOM Diabetes Prediction
   - Run BYOM Notebook 
   - Define BYOM Model 
   - Import Version
   - Evaluate
   - Review evaluation report
   - Approve 
   - Deploy 
   - Deployments/executions
   - Retire

#### View Predictions

In the next version of ModelOps, you will be able to view the predictions that follow the standard pattern directly via the UI. However, for now, we can view it here. As the same predictions table contains the predictions for all the jobs, we filter by the `airflow_job_id`. You can find this id in the UI under deployment executions.

In [4]:
import pandas as pd
from teradataml import get_connection

pd.options.display.max_colwidth = 250

airflow_job_id = "5761d5c1-bf57-456b-8076-c3062be0b544-scheduled__2022-07-11T00:00:00+00:00"

pd.read_sql(f"SELECT TOP 5 * FROM pima_patient_predictions WHERE job_id='{airflow_job_id}'", get_connection())

,job_id,PatientId,HasDiabetes,json_report


## CLI 


```bash
pip install aoa==6.1.0
```

##### Copy CLI Config

```
Copy the CLI config from ModelOps UI -> Session Details -> CLI config
```

##### Add Dataset Connection

```bash
aoa connection add
```

##### List Feature Metadata

```bash
aoa feature list-stats -m {your-db}.aoa_feature_metadata
```

##### Clone Project

```bash
aoa clone 
```

```bash
cd modelops-demo-models
```

##### Install Model Dependencies

```bash
pip install -r model_definitions/python-diabetes/model_modules/requirements.txt
```

##### Train Model

```bash
aoa run
```

##### Add Model

```bash
aoa add
```
